<a href="https://colab.research.google.com/github/MSFTserver/AI-Colab-Notebooks/blob/main/ESRGAN_4_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ESRGAN for increasing resolution ON VIDEOS!!!**
by HostsServer aka [MSFTserver](https://github.com/MSFTserver)
####**-SEEMS TO ONLY WORK CONSISTENTLY ON CHROME!!**
#### **-DOUBLE CHECK YOUR RUNTIME TYPE IS SET TO GPU**
example: an image with height 500px by width 500px scaled to an image with height 1762px by width 1762px

**You might need to go to Runtime>Restart Runtime**

If you open the folder icon (in the left menu) you will see all your files, the good thing about doing **Runtime>Restart Runtime** is that it keeps all the files you downloaded but Restarts the Memory on the Cloud Computer so you can put the ESRGAN model in memory, compared to **Runtime>Factory Reset Runtime** which Restarts the whole Cloud Computer to a default clear state.


In [ ]:
#@markdown # **1) Download required Libraries**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
from google.colab import drive, files
import os, shutil, subprocess
drive_mounted = False
temp_folder = 'tmp_frames'
result_folder = 'results'
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd ..

In [ ]:
#@markdown # **2.A) Optional:** Mount Google Drive
#@markdown uploading videos is really slow it will be faster to connect your drive and spcify the filename or path the video you want to use here

#@markdown Step 2.B can be Skipped if using google drive!
if os.path.exists('/content/drive') is not True:
  drive.mount('/content/drive')

drive_mounted = True
drive_path = 'test.mp4'#@param{type: "string"}
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive/{drive_path}'
print("video path set:\n",input_path)

In [ ]:
#@markdown # **2.B) Run this block and Upload Video using the DIALOG box!**
#@markdown Wait till it loads the video and once it's done run the next block

#@markdown You can drag and drop into choose files button
#@markdown ## SKIP this if you mounted your Google Drive!
if drive_mounted is not True:

  upload_folder = 'upload'

  if os.path.isdir(upload_folder):
      shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)
  
  # upload images
  uploaded = files.upload()
  file_name = next(iter(uploaded))
  input_path = f'/content/{upload_folder}/{file_name}'
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
else:
  print("drive mounted nothing to do here move along")

In [ ]:
#@markdown # **3) Extract Frames from Video**

if os.path.isdir(temp_folder):
  shutil.rmtree(temp_folder)

os.mkdir(temp_folder)
print(f'Extracting Frames to: {temp_folder}')
cmd = [
       'ffmpeg',
       '-i',
       input_path,
       '-qscale:v',
       '1',
       '-qmin',
       '1',
       '-qmax',
       '1',
       '-vsync',
       '0',
       f'{temp_folder}/frame_%08d.png'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    frame_count = len(os.listdir(f'/content/{temp_folder}'))
    print(f"Done, Extracted {frame_count} Frames")


In [ ]:
#@markdown # **4) Run ERSGAN on Extracted Frames!**
%cd /content/Real-ESRGAN
frame_count = len(os.listdir(f'/content/{temp_folder}'))
print(f"Enhancing {frame_count} Frames with ESRGAN...")
cmd = [
       'python',
       'inference_realesrgan.py',
       '-n',
       'RealESRGAN_x4plus',
       '-i',
       f'../{temp_folder}',
       '--outscale',
       '4',
       '--half',
       '--face_enhance'
]
#process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# stdout, stderr = process.communicate()
# if process.returncode != 0:
#     print(stderr)
#     raise RuntimeError(stderr)
# else:
#     print("Done Extracting Frames")
!{' '.join(cmd)}
print("Done Enhancing Frames")
%cd ..

In [ ]:
#@markdown # **5) Recreate Video From Enhanced Frames!**
import subprocess
frame_count = len(os.listdir(f'/content/{temp_folder}'))
if os.path.isdir(result_folder):
  shutil.rmtree(result_folder)
os.mkdir(result_folder)

fps = 15 #@param {type: "number"}
print(f"Recompiling {frame_count} Frames into Video...")
cmd = [
       'ffmpeg',
       '-i',
       f'/content/Real-ESRGAN/results/frame_%08d_out.png',
       '-c:a',
       'copy',
       '-c:v',
       'libx264',
       '-r',
       str(fps),
       '-pix_fmt',
       'yuv420p',
       f'{result_folder}/enhanced_{file_name}'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Done Recreating Video")

In [ ]:
#@markdown # **6) Download Super Enhanced Video**
#@markdown if drive is mounted this will save the file to the root of your google drive

from google.colab import files

filename= os.listdir(result_folder)
if drive_mounted:
  if os.path.exists('/content/drive/MyDrive/Hosts-AI-Edits/ESRGAN-4-Vid') is not True:
    os.makedirs('/content/drive/MyDrive/Hosts-AI-Edits/ESRGAN-4-Vid')
  shutil.move(f'{result_folder}/{filename[0]}',f'/content/drive/MyDrive/Hosts-AI-Edits/ESRGAN-4-Vid/{filename[0]}')
  print(f"moved {filename[0]} to the folder Hosts-AI-Edits/ESRGAN-4-Vid on your Google Drive")
else:
  files.download(f'{result_folder}/{filename[0]}')

In [ ]:
#@markdown # **Optional**: Unmount Google Drive
#@markdown might be useful if you decide you want to upload on here instead
drive_mounted = False
drive.flush_and_unmount()